<a href="https://colab.research.google.com/github/DrivenIdeaLab/LLM-Zero-to-Hundred/blob/master/CaesarBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
!pip install transformers
!pip install accelerate
!pip install -i https://test.pypi.org/simple/ bitsandbytes
!pip install flask-ngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.1 MB/s eta 0:00:00
Looking in indexes: https://test.pypi.org/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 MB 18.8 MB/s eta 0:00:00


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cre

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig, pipeline
import torch
from accelerate import Accelerator


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('http'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

In [ ]:
model_name="meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
pipeline = pipeline(
    "text-generation",
    model=model_name,
    torch_dtype=torch.float16,
    device_map={"": Accelerator().process_index},
    trust_remote_code=True
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
class LLM:
  def __init__(self):
    self.pipeline = pipeline
    self.tokenizer = tokenizer

  def generate(self, prompt):
    sequences = self.pipeline(
        prompt,
        do_sample=True,
        top_k=40,
        num_return_sequences=1,
        eos_token_id=self.tokenizer.eos_token_id,
        max_length=10000,
    )
    return sequences[0]['generated_text']

In [ ]:
llm = LLM()

In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://xocrdubj7j-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [ ]:
import os
import string
import random
import logging
import math

from flask import Flask, jsonify, request, send_from_directory
from flask_ngrok import run_with_ngrok


app = Flask(__name__)
run_with_ngrok(app)

@app.route('/generate', methods=['POST'])
def intiator():
  json = request.json

  prompt = json['prompt']
  print("Generating results")

  o = llm.generate(prompt)
  o = o[o.index("[/INST]")+len("[/INST]"):].strip()

  return jsonify({ "output": o })

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://f535-34-32-241-160.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
Generating results


INFO:werkzeug:127.0.0.1 - - [19/Sep/2023 05:38:53] "POST /generate HTTP/1.1" 200 -


Generating results


...

INFO:werkzeug:127.0.0.1 - - [19/Sep/2023 05:43:15] "POST /generate HTTP/1.1" 200 -


Generating results


INFO:werkzeug:127.0.0.1 - - [19/Sep/2023 05:43:40] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Sep/2023 05:44:16] "POST /generate HTTP/1.1" 200 -


Generating results
Generating results


INFO:werkzeug:127.0.0.1 - - [19/Sep/2023 05:44:34] "POST /generate HTTP/1.1" 200 -
